# Exploration du Dataset

Ce notebook sert à lancer l'analyse exploratoire des images et des données associées.

* Vérifier la structure des dossiers
* Afficher quelques exemples d'images
* Préparer un jeu de données pour l'entraînement

In [ ]:
import os
from pathlib import Path

# définir les chemins de base
base_dir = Path("..")  # assuming notebook is in ia_vision/notebooks

# on affiche la structure des dossiers data
for sub in ["raw", "processed", "splits"]:
    path = base_dir / "data" / sub
    print(f"{sub}: {path.resolve()}")

# voir si des fichiers existent
print("\nContenu du répertoire racine:")
print(list(base_dir.iterdir()))


In [ ]:
# Exemple d'affichage d'une image (si dataset présent)
from PIL import Image
import matplotlib.pyplot as plt

sample_path = base_dir / "data" / "raw"
if sample_path.exists():
    files = list(sample_path.glob("*.jpg"))
    if files:
        img = Image.open(files[0])
        plt.imshow(img)
        plt.axis('off')
        plt.title(str(files[0].name))
    else:
        print("Pas de fichiers jpg dans data/raw")
else:
    print("Dossier data/raw introuvable")


## Étapes suivantes

1. Placer les images dans `data/raw/`.
2. Ajouter du code de prétraitement dans `src/preprocess.py`.
3. Enregistrer/importer le modèle dans `src/model.py`.
4. Mettre à jour `requirements.txt` avec les dépendances utilisées.
5. Pousser les modifications vers GitHub et partager avec l'équipe.